In [88]:
from lifelines import CoxPHFitter
import seaborn as sns
import matplotlib.pyplot as plt

from src import params
import pandas as pd
from src import Models
from pathlib import Path
from src.Evaluate import metric,print_model_fits,sl_eva,evaluate_cox_pred
from src import Shap
from src import SuperLearner

# platform = "jupyter"
# params.confirm_cwd(platform)
model_params = params.model_params
colors = {'blue':'#62a0cb','orange':'#ff7f0e','green':'#2ca02c'}


In [11]:
df_eval = pd.DataFrame(columns = ['dataset','model','efron_r2','Concordance_Index','brier'])

# HRS

In [89]:
dataset = 'HRS'
df = params.data_reader(dataset=dataset,source='us',bio=False)

df['duration']=[x-y if not pd.isnull(x) else 2019-y for x,y in zip(df['deathYR'],df['interview_year'])]
predictors = model_params['domain_dict']['all']
data = df[predictors+['duration','death']]
eval_dict = {'dataset':dataset, 'model':'coxph'}

In [ ]:

correlation_matrix = data[predictors].corr()
#plt.figure(figsize=(12, 8))
#sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
#plt.show()

cols_dict = {}
for col in predictors:
    if True in list(correlation_matrix[col]>=0.8):
        matches = list(correlation_matrix.loc[correlation_matrix[col]>=0.8].index)
        values = list(correlation_matrix.loc[correlation_matrix[col]>=0.8,col].values)
        matches.remove(col)
        values.remove(1)
        if len(matches)>0:
            cols_dict[f'{col}_{matches[0]}']=values[0]
            print(f'{col}, {matches[0]},{values[0]}')

        
predictors.remove('fatherunemp')
predictors.remove('modactivityYN')
predictors.remove('Zpurpose')
predictors.remove('Zposaffect')

In [92]:
# split the dataset 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[predictors],data[['duration','death']],test_size=0.4,
                                                                                random_state=model_params['random_state'])

In [111]:
X_test['interview_year']=df.loc[X_test.index,'interview_year']

In [112]:
data = pd.concat([X_train,y_train], axis=1)
cph = CoxPHFitter()
cph.fit(data, duration_col='duration', event_col='death')
# producing predictions 
survival_probabilities_06 = cph.predict_survival_function(X_test, times=2019-2006)
survival_probabilities_08 = cph.predict_survival_function(X_test, times=2019-2008)


In [122]:
survival_probabilities = [x if y==2006 else z for x,y,z in zip(np.array(survival_probabilities_08),X_test['interview_year'],np.array(survival_probabilities_08))]


In [134]:
eval_dict = {**eval_dict, **evaluate_cox_pred(np.array(y_test['death']), np.array(survival_probabilities))}
eval_dict['Concordance_Index']=cph.concordance_index_
df_eval = pd.concat([df_eval, pd.DataFrame(eval_dict,index=[0])],axis=0)

In [93]:
import numpy as np
import pandas as pd
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored

train_y =np.array([(row['death'], row['duration']) for index, row in data.iterrows()], dtype=[('event', '?'), ('time', '<f8')])  
train_x = data[predictors]

# Initialize RSF model
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, max_features="sqrt", n_jobs=-1, random_state=model_params['random_state'])


In [94]:
# Train the model
rsf.fit(train_x, train_y)

RandomSurvivalForest(min_samples_leaf=15, min_samples_split=10, n_jobs=-1,
                     random_state=87785)

In [142]:
prediction = rsf.predict(X_test.drop(columns=['interview_year']))
event_indicator = [True if x ==1 else False for x in y_test['death']]
event_time = y_test['duration']

c_index = concordance_index_censored(event_indicator, event_time, prediction)
print("Concordance Index:", c_index[0])


survival_probabilities = rsf.predict_survival_function(X_test.drop(columns=['interview_year']))
probabilities_at_timepoint = [fn(2019-2006) if x == 2006 else fn(2019-2008) for fn,x in zip(survival_probabilities,X_test['interview_year'])]


eval_dict = {'dataset':dataset,'model':'random survival forest','Concordance_Index':c_index[0]}
eval_dict = {**eval_dict, **evaluate_cox_pred(np.array(y_test['death']), probabilities_at_timepoint)}


Concordance Index: 0.8608929759205741


In [143]:
eval_dict

{'dataset': 'HRS',
 'model': 'random survival forest',
 'Concordance_Index': 0.8608929759205741,
 'efron_r2': -1.4817293715630258,
 'brier': 0.5283411432225518}

In [144]:
df_eval = pd.concat([df_eval, pd.DataFrame(eval_dict,index=[0])],axis=0)
df_eval

,dataset,model,efron_r2,Concordance_Index,brier
0,SHARE,random survival forest,-3.087881,0.865517,0.636617
1,SHARE,coxph,-3.060498,0.783441,0.632352
2,ELSA,coxph,-15.983848,0.805159,0.858478
3,ELSA,random survival forest,-16.068137,0.865035,0.862738
0,HRS,coxph,-1.593842,0.771599,0.552209
0,HRS,random survival forest,-1.481729,0.860893,0.528341


# SHARE

In [42]:
dataset='SHARE'
df = params.data_reader(dataset='SHARE',source='us',bio=False)
df = df.loc[(df['deathY']>2005)|( df['death']==0),]

domain_name = 'share_all'
model_params['domain_dict'][domain_name]= list(set(model_params['domain_dict']['all']).intersection(set(df.columns)))


df['duration']=[x-2006 if not pd.isnull(x) else 2021-2006 for x in df['deathY']]
predictors = model_params['domain_dict'][domain_name]
data = df[predictors+['duration','death']]



correlation_matrix = data[predictors].corr()
#plt.figure(figsize=(12, 8))
#sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
#plt.show()

# split the dataset 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[predictors],data[['duration','death']],test_size=0.4,
                                                                                random_state=model_params['random_state'])

In [69]:
# COXPH
eval_dict = {'dataset':dataset, 'model':'coxph'}

data = pd.concat([X_train,y_train], axis=1)
cph = CoxPHFitter()
cph.fit(data, duration_col='duration', event_col='death')
# producing predictions 
survival_probabilities = cph.predict_survival_function(X_test, times=2021-2006)

eval_dict = {**eval_dict, **evaluate_cox_pred(np.array(y_test['death']), np.array(survival_probabilities))}
eval_dict['Concordance_Index']=cph.concordance_index_

df_eval = pd.concat([df_eval, pd.DataFrame(eval_dict,index=[0])],axis=0)

In [44]:
# random forest 
train_y =np.array([(row['death'], row['duration']) for index, row in data.iterrows()], dtype=[('event', '?'), ('time', '<f8')])  
train_x = data[predictors]

# Initialize RSF model
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, max_features="sqrt", n_jobs=-1, random_state=model_params['random_state'])

# Train the model
rsf.fit(train_x, train_y)

RandomSurvivalForest(min_samples_leaf=15, min_samples_split=10, n_jobs=-1,
                     random_state=87785)

In [56]:
prediction = rsf.predict(X_test.values)
event_indicator = [True if x ==1 else False for x in y_test['death']]
event_time = y_test['duration']

c_index = concordance_index_censored(event_indicator, event_time, prediction)
print("Concordance Index:", c_index[0])

survival_probabilities = rsf.predict_survival_function(X_test)
probabilities_at_timepoint = [fn(2021-2006) for fn in survival_probabilities]


eval_dict = {'dataset':dataset,'model':'random survival forest','Concordance_Index':c_index[0]}
eval_dict = {**eval_dict, **evaluate_cox_pred(np.array(y_test['death']), probabilities_at_timepoint)}
df_eval = pd.concat([df_eval, pd.DataFrame(eval_dict,index=[0])],axis=0)
df_eval

X does not have valid feature names, but RandomSurvivalForest was fitted with feature names


Concordance Index: 0.865517129383793


,dataset,model,efron_r2,Concordance_Index,brier
0,HRS,coxph,-1.810933,0.771599,0.598426
0,HRS,random survival forest,-10009.632719,0.860893,2131.186904
0,SHARE,random survival forest,-56.807512,0.865517,9.002520
0,SHARE,coxph,-3.060498,0.783441,0.632352
0,SHARE,random survival forest,-3.087881,0.865517,0.636617


# ELSA

In [75]:
dataset = 'ELSA'
df = params.data_reader(dataset=dataset,source='us',bio=False)


domain_name = 'elsa_all'
# recode a new domain dict for SHARE  based on its columns
model_params['domain_dict'][domain_name]= list(set(model_params['domain_dict']['all']).intersection(set(df.columns)))



df['duration']=[x-2005 if not pd.isnull(x) else 2012-2005 for x in df['radyear']]
predictors = model_params['domain_dict'][domain_name]
predictors.remove('sleepYN')
data = df[predictors+['duration','death']]






correlation_matrix = data[predictors].corr()
#plt.figure(figsize=(12, 8))
#sns.heatmap(correlation_matrix, annot=False, cmap="coolwarm", vmin=-1, vmax=1)
#plt.show()

# split the dataset 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data[predictors],data[['duration','death']],test_size=0.4,
                                                                                random_state=model_params['random_state'])

In [78]:
# COXPH
eval_dict = {'dataset':dataset, 'model':'coxph'}

data = pd.concat([X_train,y_train], axis=1)
cph = CoxPHFitter()
cph.fit(data, duration_col='duration', event_col='death')
# producing predictions 
survival_probabilities = cph.predict_survival_function(X_test, times=2012-2005)

eval_dict = {**eval_dict, **evaluate_cox_pred(np.array(y_test['death']), np.array(survival_probabilities))}
eval_dict['Concordance_Index']=cph.concordance_index_

df_eval = pd.concat([df_eval, pd.DataFrame(eval_dict,index=[0])],axis=0)

In [81]:
# random forest 
train_y =np.array([(row['death'], row['duration']) for index, row in data.iterrows()], dtype=[('event', '?'), ('time', '<f8')])  
train_x = data[predictors]

# Initialize RSF model
rsf = RandomSurvivalForest(n_estimators=100, min_samples_split=10, min_samples_leaf=15, max_features="sqrt", n_jobs=-1, random_state=model_params['random_state'])

# Train the model
rsf.fit(train_x, train_y)

RandomSurvivalForest(min_samples_leaf=15, min_samples_split=10, n_jobs=-1,
                     random_state=87785)

In [82]:
prediction = rsf.predict(X_test)
event_indicator = [True if x ==1 else False for x in y_test['death']]
event_time = y_test['duration']

c_index = concordance_index_censored(event_indicator, event_time, prediction)
print("Concordance Index:", c_index[0])

survival_probabilities = rsf.predict_survival_function(X_test)
probabilities_at_timepoint = [fn(2012-2005) for fn in survival_probabilities]


eval_dict = {'dataset':dataset,'model':'random survival forest','Concordance_Index':c_index[0]}
eval_dict = {**eval_dict, **evaluate_cox_pred(np.array(y_test['death']), probabilities_at_timepoint)}
df_eval = pd.concat([df_eval, pd.DataFrame(eval_dict,index=[0])],axis=0)
df_eval

Concordance Index: 0.8650352678372701


X does not have valid feature names, but RandomSurvivalForest was fitted with feature names


,dataset,model,efron_r2,Concordance_Index,brier
0,HRS,coxph,-1.810933,0.771599,0.598426
1,HRS,random survival forest,-10009.632719,0.860893,2131.186904
2,SHARE,random survival forest,-3.087881,0.865517,0.636617
0,SHARE,coxph,-3.060498,0.783441,0.632352
0,ELSA,coxph,-15.983848,0.805159,0.858478
0,ELSA,random survival forest,-16.068137,0.865035,0.862738


In [145]:
df_eval.to_csv(Path.cwd()/'results/survival_models.csv',index=False)

In [147]:
print(df_eval.to_latex(index=False, caption='Results of Survival Models for three datasets', label='tab:survival_res'))

\begin{table}
\centering
\caption{Results of Survival Models for three datasets}
\label{tab:survival_res}
\begin{tabular}{llrrr}
\toprule
dataset &                  model &   efron\_r2 &  Concordance\_Index &    brier \\
\midrule
  SHARE & random survival forest &  -3.087881 &           0.865517 & 0.636617 \\
  SHARE &                  coxph &  -3.060498 &           0.783441 & 0.632352 \\
   ELSA &                  coxph & -15.983848 &           0.805159 & 0.858478 \\
   ELSA & random survival forest & -16.068137 &           0.865035 & 0.862738 \\
    HRS &                  coxph &  -1.593842 &           0.771599 & 0.552209 \\
    HRS & random survival forest &  -1.481729 &           0.860893 & 0.528341 \\
\bottomrule
\end{tabular}
\end{table}



In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
